<a href="https://colab.research.google.com/github/thatjasonr/video-scraper/blob/main/Leeds_VIdeos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 30.3 MB/s eta 0:00:00
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,375 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,798 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,753 kB]
Get:11 https://ppa.launchpadcontent

In [3]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install

--2025-03-15 04:09:55--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 173.194.174.136, 173.194.174.93, 173.194.174.190, ...
Connecting to dl.google.com (dl.google.com)|173.194.174.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114745904 (109M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 109.43M   263MB/s    in 0.4s    

2025-03-15 04:09:55 (263 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [114745904/114745904]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 125509 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (134.0.6998.88-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Packa

In [4]:
!wget https://chromedriver.storage.googleapis.com/134.0.6998.88/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!mv chromedriver /usr/bin/chromedriver


--2025-03-15 04:10:23--  https://chromedriver.storage.googleapis.com/134.0.6998.88/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 173.194.174.207, 74.125.23.207, 74.125.203.207, ...
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|173.194.174.207|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-03-15 04:10:24 ERROR 404: Not Found.

unzip:  cannot find or open chromedriver_linux64.zip, chromedriver_linux64.zip.zip or chromedriver_linux64.zip.ZIP.
mv: cannot stat 'chromedriver': No such file or directory


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# Set up Chrome options for headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Initialize WebDriver
driver = webdriver.Chrome(options=chrome_options)

def fetch_all_video_data():
    driver.get("https://video.leedsunited.com/view?search=Latest%20Videos")

    # Wait for video cards to load
    videos = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "card-video-container"))
    )

    video_data = []

    for video in videos:
        try:
            # Extract runtime
            runtime = video.find_element(By.CLASS_NAME, "runtime").text

            # Extract title
            title = video.find_element(By.CLASS_NAME, "card-body__title").get_attribute("title")

            # Extract categories
            categories_div = video.find_element(By.CLASS_NAME, "card-body__categories")
            categories = [category.text for category in categories_div.find_elements(By.CLASS_NAME, "card-body__category")]

            # Extract image URL
            image_element = video.find_element(By.TAG_NAME, "img")
            image_url = image_element.get_attribute("src")

            # Extract entitlement (e.g., "premium" or "free")
            try:
                entitlement = video.find_element(By.CLASS_NAME, "entitlement").text
            except:
                entitlement = "free"  # Default to "free" if not found

            # Extract video URL
            try:
                parent_a_tag = video.find_element(By.XPATH, "./ancestor::a")  # Get the enclosing <a> tag
                video_url = parent_a_tag.get_attribute("href")
                if not video_url.startswith("http"):  # Handle relative URLs
                    video_url = f"https://video.leedsunited.com{video_url}"
            except:
                video_url = "N/A"  # Default if URL is not found

            # Add data to the list
            video_data.append({
                "title": title,
                "runtime": runtime,
                "categories": categories,
                "image_url": image_url,
                "entitlement": entitlement,
                "video_url": video_url,  # Add URL to the data structure
            })

        except Exception as e:
            print(f"Skipped a video due to error: {e}")

    return video_data

# Fetch video data
data = fetch_all_video_data()

# Process the data (filter and print)
filtered_data = [video for video in data if "Full Match Replay" in video['title']]

# Print the filtered data
for video in filtered_data:
    print(f"Title: {video['title']}")
    print(f"Runtime: {video['runtime']}")
    print(f"Categories: {', '.join(video['categories'])}")
    print(f"Image URL: {video['image_url']}")
    print(f"Entitlement: {video['entitlement']}")
    print(f"Video URL: {video['video_url']}")
    print("-" * 40)

# Close the browser
driver.quit()


Title: Full Match Replay: Leeds United v Millwall
Runtime: 01:43:31
Categories: FIRST TEAM, MATCH ACTION
Image URL: https://video.leedsunited.com/_assets/images/placeholder.png
Entitlement: PREMIUM
Video URL: https://video.leedsunited.com/video/dm9kJTdDMF83NmtjdXVscCU3QyUyRnBhZ2UlM0ZzbHVnJTNEaG9tZSU3Q2hvbWUlMkZsaXN0JTJGMCUyRmZlZWQ%3D
----------------------------------------
Title: Full Match Replay: Aldershot Town v Leeds United U21s
Runtime: 01:41:32
Categories: UNDER 21’S, MATCH ACTION
Image URL: https://video.leedsunited.com/_assets/images/placeholder.png
Entitlement: 
Video URL: https://video.leedsunited.com/video/dm9kJTdDMF93bHRuODdwciU3QyUyRnBhZ2UlM0ZzbHVnJTNEaG9tZSU3Q2hvbWUlMkZsaXN0JTJGMCUyRmZlZWQ%3D
----------------------------------------
Title: Full Match Replay: Leeds United Women 3-0 Norton & Stockton Ancients
Runtime: 01:33:38
Categories: WOMENS, MATCH ACTION
Image URL: https://video.leedsunited.com/_assets/images/placeholder.png
Entitlement: 
Video URL: https://video.leed